# Use inceptionv4 and multiple input

# 1. Data Clean

In [4]:
#Define data clean function
# if you need to do zero center, you should use data_normal rather than data_clean.

import matplotlib.pyplot as plt
def plt_images(X_images):
    plt.imshow(X_images,cmap = plt.cm.gray)

def data_normal(X_,y_,is_training):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images_1 = []
    images_2 = []
    avergae_bright=[]
    index=1
    for i, row in X_.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
#         avei=(np.sum(band_1)+np.sum(band_2))/(band_1.shape[0]+band_2.shape[0])
        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        
        images_1.append(r[:,:,np.newaxis])
        images_2.append(g[:,:,np.newaxis])
#         avergae_bright.append(avei)
        
    X_data_angle=np.reshape(np.sin(X_["inc_angle"]),(X_["inc_angle"].shape[0],1))
    X_data_plus=np.concatenate([X_data_angle[:, np.newaxis]], 
                              axis=2)
    X_data_plus=X_data_plus.reshape(X_data_plus.shape[0],X_data_plus.shape[2])
    if is_training:
        y_data = keras.utils.to_categorical(y_, num_classes)
    else:
        y_data = None

    return (np.array(images_1),np.array(images_2),X_data_plus,y_data)


# Define threadsafe_generator
# If we want to use data augmentation in training process,we have to use @threadsafe_generator to make any generator thread safe.

# Note: A non thread safe generator in a multithreaded envrionment just crashes, yieldsing a 'ValueError: generator already executing' error.You can try it yourself by removing the @threadsafe_generator decorator from count().

# Also, since we have to use multiple generators to feed both the image data and the angle data, we have to define our own generator: generator_img_angle()
import threading
from keras.preprocessing.image import ImageDataGenerator

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()
 
    def __iter__(self):
        return self
 
    def __next__(self):
        with self.lock:
            return self.it.__next__()
 
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g
 
@threadsafe_generator
def generator_img_plus_new( generator, X1,X2,X_plus, y, batch_size = 32 ):
    SEED=816
    generator_seed = np.random.randint( SEED )
    gen_X1 = generator.flow( X1, y, 
                             batch_size = batch_size, seed = generator_seed )
    gen_X2 = generator.flow( X1, X2, 
                             batch_size = batch_size, seed = generator_seed )
    gen_X3 = generator.flow( X1, X_plus, 
                             batch_size = batch_size, seed = generator_seed )

    while True:
        X1i = gen_X1.next()
        X2i = gen_X2.next()
        X3i = gen_X3.next()
        yield [ X1i[0], X2i[1],X3i[1] ], X1i[1]
    
#load data
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import keras
import math
#Load the data.
train = pd.read_json("../input/train.json")

#use mean of angle to replace "na"
inc_angle = train.inc_angle.replace('na',0)
idx=np.where(inc_angle==0)
inc_angle = inc_angle.drop(idx[0])
inc_mean=np.mean(inc_angle)
train.inc_angle = train.inc_angle.replace('na',inc_mean)


#Do Data Cleaning on training data and validation data
num_classes=2


y_train=train['is_iceberg']

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(train,y_train, random_state=1, train_size=0.75)
# X_train_cv,X_train_angle,y_train_cv=data_clean(X_train_cv,y_train_cv,is_training=True)
# X_valid,X_valid_angle,y_valid=data_clean(X_valid,y_valid,is_training=True)

X_train_1,X_train_2,X_train_plus,y_train_cv=data_normal(X_train_cv,y_train_cv,is_training=True)
X_valid_1,X_valid_2,X_valid_plus,y_valid=data_normal(X_valid,y_valid,is_training=True)

print(X_train_1.shape,X_train_2.shape,X_train_plus.shape,y_train_cv.shape)
print(X_valid_1.shape,X_valid_2.shape,X_valid_plus.shape,y_valid.shape)

/home/cv/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(1203, 75, 75, 1) (1203, 75, 75, 1) (1203, 1) (1203, 2)
(401, 75, 75, 1) (401, 75, 75, 1) (401, 1) (401, 2)


/home/cv/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


# 2. Train model

If we use data augmentaion, use the model.fit_generator version, otherwise use model.fit

In [10]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
import keras
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalMaxPooling2D
import os
from keras import regularizers
import math
from keras.layers import advanced_activations

#Macro defined
batch_size =32
num_classes =2
epochs = 200
batch_num_per_epoch=math.ceil(X_train_cv.shape[0]/batch_size)
batch_num_val=math.ceil(X_valid.shape[0]/batch_size)
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_iceberg_myownmodel2.h5'
filepath="keras_iceberg_epoch_mymodel.h5"

# create the base model
def conv2d_block(inputs,kernel_size,filters):
    x=Conv2D(filters=filters,kernel_size= kernel_size,strides=(1, 1), padding='same',
           input_shape=X_train_cv.shape[1:],kernel_initializer='glorot_normal',
           use_bias=True,kernel_regularizer=regularizers.l2(0.05))(inputs)
    
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, 
                                scale=True, beta_initializer='zeros', gamma_initializer='ones', 
                                moving_mean_initializer='zeros', moving_variance_initializer='ones')(x)
#     x=advanced_activations.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None)(x)
    x=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Dropout(0.3)(x)
    return x

def dense_block(x,filters):
    x=Dense(filters,kernel_initializer='glorot_normal',kernel_regularizer=regularizers.l2(0.05))(x)
    x=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
    x=Dropout(0.1)(x)
    return x

def model_i(input_x,angle_input):
    x= conv2d_block(input_x,(5,5),64)
    x= conv2d_block(x,(3,3),128)
    x= conv2d_block(x,(3,3),64)
    image_out=GlobalMaxPooling2D()(x)
    x = keras.layers.concatenate([image_out, angle_input],axis=-1)
#     x=dense_block(x,64)
    x=dense_block(x,16)
    prediction = Dense(num_classes, activation='relu',kernel_initializer='glorot_normal',
                   kernel_regularizer=regularizers.l2(0.05))(x)
    return prediction

main_input1 = Input(shape=X_train_1.shape[1:],name='image_input1')
main_input2 = Input(shape=X_train_2.shape[1:],name='image_input2')
angle_input = Input(shape=(1,), name='angle_input')

x1_pre=model_i(main_input1,angle_input)
x2_pre=model_i(main_input2,angle_input)

x_pre=keras.layers.concatenate([x1_pre,x2_pre])
# x_pre=dense_block(x_pre,256)
# x_pre=dense_block(x_pre,16)

predictions = Dense(num_classes, activation='softmax',name='main_output',
                    kernel_regularizer=regularizers.l2(0.01),
                    kernel_initializer='glorot_normal')(x_pre)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[main_input1,main_input2, angle_input], outputs=predictions)

opt=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Let's train the model using RMSprop
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

#input data

# Fit the model on the batches generated by datagen.flow().
# model.fit({'image_input1': X_train_1,'image_input2': X_train_2, 'angle_input':  X_train_plus},
#           {'main_output': y_train_cv},
#           validation_data=([X_valid_1,X_valid_2,X_valid_plus],y_valid),
#           epochs=epochs, batch_size=batch_size,
#           callbacks=[keras.callbacks.ModelCheckpoint(filepath,
#                                                         monitor='val_acc',
#                                                         verbose=0,
#                                                         save_best_only=True, 
#                                                         mode='auto')])
image_augmentation = ImageDataGenerator( rotation_range = 20,
                                         horizontal_flip = True,
                                         vertical_flip = True,
                                         width_shift_range = .3,
                                         height_shift_range =.3,
                                         zoom_range = .1 )

train_generator = generator_img_plus_new( image_augmentation, X_train_1,X_train_2,
                                X_train_plus, y_train_cv, 
                                batch_size = batch_size)
model.fit_generator(train_generator,epochs=epochs,
                    validation_data=([X_valid_1,X_valid_2,X_valid_plus],y_valid),
#                   validation_data=generator_img_angle(X_valid,X_valid_angle,y_valid,
#                                                       batch_size,is_training=True),
                    validation_steps=batch_num_val,
                    workers=4,
                    steps_per_epoch=batch_num_per_epoch,
                    # 该回调函数将在每个epoch后保存模型到filepath
                    callbacks=[keras.callbacks.ModelCheckpoint(filepath,
                                                               monitor='val_acc',
                                                               verbose=0,
                                                               save_best_only=True, 
                                                               mode='auto')])
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate([X_valid_1,X_valid_2,X_valid_plus],y_valid, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/200
38/38 [==============================] - 6s 169ms/step - loss: 15.6699 - acc: 0.5936 - val_loss: 14.8000 - val_acc: 0.5312
Epoch 2/200
38/38 [==============================] - 3s 90ms/step - loss: 14.0486 - acc: 0.6005 - val_loss: 13.3138 - val_acc: 0.5312
Epoch 3/200
38/38 [==============================] - 3s 89ms/step - loss: 12.6322 - acc: 0.6353 - val_loss: 12.0117 - val_acc: 0.5312
Epoch 4/200
38/38 [==============================] - 4s 92ms/step - loss: 11.3939 - acc: 0.6285 - val_loss: 10.8691 - val_acc: 0.5312
Epoch 5/200
38/38 [==============================] - 4s 92ms/step - loss: 10.3356 - acc: 0.6115 - val_loss: 9.8673 - val_acc: 0.5312
Epoch 6/200
38/38 [==============================] - 4s 94ms/step - loss: 9.3703 - acc: 0.6216 - val_loss: 8.9860 - val_acc: 0.5312
Epoch 7/200
38/38 [==============================] - 3s 91ms/step - loss: 8.5472 - acc: 0.6030 - val_loss: 8.2099 - val_acc: 0.5312
Epoch 8/200
38/38 [==============================] - 3s 91ms/step 

38/38 [==============================] - 3s 89ms/step - loss: 0.4505 - acc: 0.9641 - val_loss: 0.9171 - val_acc: 0.7456
Epoch 124/200
38/38 [==============================] - 3s 89ms/step - loss: 0.4785 - acc: 0.9465 - val_loss: 0.7085 - val_acc: 0.8130
Epoch 125/200
38/38 [==============================] - 3s 89ms/step - loss: 0.4465 - acc: 0.9641 - val_loss: 0.7300 - val_acc: 0.7905
Epoch 126/200
38/38 [==============================] - 3s 88ms/step - loss: 0.4402 - acc: 0.9638 - val_loss: 0.6425 - val_acc: 0.8678
Epoch 127/200
38/38 [==============================] - 3s 89ms/step - loss: 0.4373 - acc: 0.9589 - val_loss: 0.6442 - val_acc: 0.8579
Epoch 128/200
38/38 [==============================] - 3s 89ms/step - loss: 0.4255 - acc: 0.9704 - val_loss: 0.6836 - val_acc: 0.8155
Epoch 129/200
38/38 [==============================] - 3s 89ms/step - loss: 0.4197 - acc: 0.9723 - val_loss: 0.7396 - val_acc: 0.7880
Epoch 130/200
38/38 [==============================] - 3s 89ms/step - loss: 

## validation check

In [7]:
import keras
# Score trained model.
scores = model.evaluate([X_valid_1,X_valid_2, X_valid_plus],y_valid, verbose=1)
# batch_size=32
# batch_num_val=math.ceil(X_valid.shape[0]/batch_size)
# scores = model.evaluate_generator(generator_img_plus(X_valid, X_valid_plus, y_valid,batch_size=batch_size),
#                                   steps=batch_num_val)

print("validation accuracy:",scores)

401/401 [==============================] - 2s 4ms/step
validation accuracy: [1.1274703568651194, 0.887780549074349]


# 3.Predict

## Load a pretrained model

In [6]:
from keras.models import load_model
model_path="../iceberg_code/keras_iceberg_epoch_mymodel.h5"
model = load_model(model_path)  

## predict

In [15]:
import json
import numpy as np
import pandas as pd
import math

cut_set_num=10
lenth_test=8424
num_classes=2
is_training=False
test_path="../input/test_cut/"
batch_size=32


predicted_test=[]
result_max=[]
test_id=[]


for ci in range(cut_set_num):
    test_pathi=test_path+"test%d.json"%ci
    test_json_str = open(test_pathi, 'r').readlines()[0]
    test=pd.read_json(test_json_str)
    
    #use mean of angle to replace "na"
    inc_angle = test.inc_angle.replace('na',0)
    idx=np.where(inc_angle==0)
    inc_angle = inc_angle.drop(idx[0])
    inc_mean=np.mean(inc_angle)
    test.inc_angle = test.inc_angle.replace('na',inc_mean)
#     test= test.drop(idx[0])
#     test.inc_angle = test.inc_angle.replace('na',0)
   
    X_test_1,X_test_2,X_test_plus,y_test=data_normal(X_=test,y_=None,is_training=False)
    batch_num_predic=math.ceil(X_test_1.shape[0]/batch_size)
#     print(X_test.shape[0],batch_num_predic)
    #发现一个问题,steps总是比我设置的多出10
    predicted_testi=model.predict([X_test_1,X_test_2,X_test_plus])
    
#     predicted_testi=model.predict_generator(generator_img_angle(X11=X_test,
#                                                                 X2=X_test_angle, 
#                                                                 Y=None,
#                                                                 batch_size=batch_size,
#                                                                 is_training=False),
#                                             steps=batch_num_predic)
    
    test_id.extend(test["id"])
    predicted_testi=predicted_testi.reshape(predicted_testi.shape[0],2)
    for index in range(len(predicted_testi)):
        predicted_test.append(predicted_testi[index][1])
    result_max.extend( np.argmax(predicted_testi, axis = 1) )
    
    print("batch%d has finished!"%ci)
    print()

/home/cv/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


batch0 has finished!

batch1 has finished!

batch2 has finished!

batch3 has finished!

batch4 has finished!

batch5 has finished!

batch6 has finished!

batch7 has finished!

batch8 has finished!

batch9 has finished!



In [16]:
# save result
submission = pd.DataFrame()
print(len(test_id),len(result_max),len(predicted_test))
submission['id']=test_id
submission['is_iceberg']=result_max
submission.to_csv('sub.csv', index=False)
# save probablity

submission['is_iceberg']=predicted_test
submission.to_csv('aug_plus_my_merge_model.csv', index=False,float_format="%.6lf")


8424 8424 8424


In [31]:
print(base_model_1.summary())
from keras.utils import plot_model
plot_model(base_model_1, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1342 (Conv2D)            (None, None, None, 3 864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1222 (Batch (None, None, None, 3 96          conv2d_1342[0][0]                
__________________________________________________________________________________________________
activation_1339 (Activation)    (None, None, None, 3 0           batch_normalization_1222[0][0]   
__________________________________________________________________________________________________
conv2d_134

__________________________________________________________________________________________________
block8_5_conv (Conv2D)          (None, None, None, 2 933920      block8_5_mixed[0][0]             
__________________________________________________________________________________________________
block8_5 (Lambda)               (None, None, None, 2 0           block8_4_ac[0][0]                
                                                                 block8_5_conv[0][0]              
__________________________________________________________________________________________________
block8_5_ac (Activation)        (None, None, None, 2 0           block8_5[0][0]                   
__________________________________________________________________________________________________
conv2d_1526 (Conv2D)            (None, None, None, 1 399360      block8_5_ac[0][0]                
__________________________________________________________________________________________________
batch_norm

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.